# PhoneBook v2.0

1. Based on previous task 'PhoneBook' implement following, using functions and stored procedures:
 1) Function that returns all records based on a pattern (example of pattern: part of name, surname, phone number).
 2) Create procedure to insert new user by name and phone, update phone if user already exists.
 3) Create procedure to insert many new users by list of name and phone. Use loop and if statement in stored procedure. Check correctness of phone in procedure and return all incorrect data.
 4) Create function to querying data from the tables with pagination (by limit and offset).
 5) Implement procedure to deleting data from tables by username or phone.

In [14]:
import psycopg2
import csv
from tabulate import tabulate

# Database connection
conn = psycopg2.connect(
    host="localhost",
    dbname="mydb",
    user="postgres",
    password="240407",
    port="5432"
)
conn.autocommit = True
cur = conn.cursor()

def insert_game():
    game_id = int(input("Enter game ID: "))
    player_name = input("Enter player name: ")
    score = int(input("Enter score: "))
    duration = int(input("Enter duration in seconds: "))
    game_date = input("Enter game date (YYYY-MM-DD): ")
    cur.execute("CALL upsert_game(%s, %s, %s, %s, %s)", (game_id, player_name, score, duration, game_date))
    print("Game inserted or updated.")

def insert_from_csv():
    filepath = input("Enter CSV file path: ")
    game_list = []
    with open(filepath, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for row in reader:
            if len(row) == 5:
                game_list.append(row)
    cur.execute("CALL insert_many_games(%s)", (game_list,))
    print("Batch insert executed.")

def search_pattern():
    pattern = input("Enter search pattern: ")
    cur.execute("SELECT * FROM find_games_by_pattern(%s)", (pattern,))
    rows = cur.fetchall()
    print(tabulate(rows, headers=["Game ID", "Player", "Score", "Duration", "Date"], tablefmt="grid"))

def paginate():
    limit = int(input("Enter number of results to show: "))
    offset = int(input("Enter offset: "))
    cur.execute("SELECT * FROM get_games_page(%s, %s)", (limit, offset))
    rows = cur.fetchall()
    print(tabulate(rows, headers=["Game ID", "Player", "Score", "Duration", "Date"], tablefmt="grid"))

def delete_game():
    game_id = input("Enter game ID to delete (or leave blank): ") or None
    player_name = input("Enter player name to delete (or leave blank): ") or None
    cur.execute("CALL delete_game(%s, %s)", (game_id, player_name))
    print("Game(s) deleted if matched.")

def show_all():
    cur.execute("SELECT * FROM snake_game")
    rows = cur.fetchall()
    print(tabulate(rows, headers=["Game ID", "Player", "Score", "Duration", "Date"], tablefmt="fancy_grid"))

def main():
    while True:
        print("""
Available commands:
[i] Insert a game manually
[csv] Insert games from CSV
[s] Show all games
[q] Query games by pattern
[p] Paginate results
[d] Delete game by ID or player
[x] Exit
        """)
        cmd = input("Enter command: ").lower()

        if cmd == "i":
            insert_game()
        elif cmd == "csv":
            insert_from_csv()
        elif cmd == "s":
            show_all()
        elif cmd == "q":
            search_pattern()
        elif cmd == "p":
            paginate()
        elif cmd == "d":
            delete_game()
        elif cmd == "x":
            break
        else:
            print("Unknown command.")

    cur.close()
    conn.close()

if __name__ == '__main__':
    main()


Available commands:
[i] Insert a game manually
[csv] Insert games from CSV
[s] Show all games
[q] Query games by pattern
[p] Paginate results
[d] Delete game by ID or player
[x] Exit
        
╒═══════════╤══════════╤═════════╤════════════╤═════════════════════╕
│   Game ID │ Player   │   Score │   Duration │ Date                │
╞═══════════╪══════════╪═════════╪════════════╪═════════════════════╡
│        21 │ Test1    │     100 │         50 │ 2024-04-10 00:00:00 │
├───────────┼──────────┼─────────┼────────────┼─────────────────────┤
│        22 │ Test2    │     100 │         50 │ 2024-04-10 00:00:00 │
╘═══════════╧══════════╧═════════╧════════════╧═════════════════════╛

Available commands:
[i] Insert a game manually
[csv] Insert games from CSV
[s] Show all games
[q] Query games by pattern
[p] Paginate results
[d] Delete game by ID or player
[x] Exit
        
